In [1]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin 


  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-1hg761c_
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-1hg761c_
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=118bc46a8dd7694d060d11043084627d059f6f03db78b32b9b178c6ae51ce57f
  Stored in directory: /tmp/pip-ephem-wheel-cache-ymxwgri0/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [4]:
%%cu 
/* AUTHOR: SAGNIK CHATTERJEE */

#include <stdlib.h>
#include <stdio.h>

#define MASK_LENGTH 7 //I.E m SIZE 

//allocate constant meory 

__constant__ int MASK[MASK_LENGTH];

__global__ void convulation(int *input ,int *output,int n){
    //calculate the global thread ID 
    int tid = blockIdx.x * blockDim.x + threadIdx.x; 
    //boundary checks 
    if(tid < n){
        int temp=0;
        int radius  = MASK_LENGTH /2 ;
        int start = tid - radius; //shift it so the mask i
        
        //iterate over the length of the mask 
        for(int i=0;i<MASK_LENGTH;i++){
            //multiply where the corresspoidng massk it and sum all the products together 
            //check if we are off the wither edge of the array 
            if(start+i >=0 && start+i< n){
                //accumulate each product
                temp+=MASK[i] * input[start+i] ;
                printf("%d\n",temp);
            }
        }
        //write back the results 
        output[tid] = temp;
    }

}

//intilaize array with random numbers
void init_array(int *a,int n){
    for(int i=0;i<n;i++){
        a[i] =rand()%500;
    }
}


int main(){
      //1d conv 
      //set the input array size 
      int n = 1 << 12 ;// (2^12 elements)
      size_t bytes = n* sizeof(int); 

      //allocate space for input and output 
      int *input , *output;
      cudaMallocManaged(&input,bytes);
      cudaMallocManaged(&output,bytes);

      //init input array 
      init_array(input,n);

      //allocate space for the mask on the host
      int *mask = new int[MASK_LENGTH];
      init_array(mask,MASK_LENGTH);

      // copy the mask to the GPU
      cudaMemcpyToSymbol(MASK,mask,MASK_LENGTH * sizeof(int));

      //set and Grid dim
      int THREADS = 4;

      int BLOCKS = (n+THREADS-1)/THREADS;

      //launch kernel
      convulation<<<BLOCKS,THREADS>>>(input,output,n);

      cudaDeviceSynchronize();

      cudaFree(input);
      cudaFree(output);

      return 0;  
}

Streaming output truncated to the last 5000 lines.
124236
138639
116862
138514
117243
228080
247446
245859
306585
304936
249096
270221
290630
331187
398945
513238
485793
608283
525580
524607
589737
557073
549944
515697
442083
303504
237800
219368
148952
393217
425709
412643
392609
360465
455175
454701
467377
182044
255592
329749
282764
333542
418402
390169
412132
265814
203351
221345
238490
292838
239887
377029
265558
282084
374736
420047
409078
244856
317234
292996
255009
313089
225596
129832
79161
283939
266482
363966
403688
287312
230106
204615
351073
251560
300862
326004
324970
254324
226617
169323
231483
337053
323663
298796
242441
387090
420706
381179
379938
266066
324475
340232
278458
223766
217950
285744
283262
315973
247904
306659
259651
277980
190268
147233
204359
260760
242225
236444
261050
222338
193965
142267
196367
292365
205256
115860
146953
191451
292061
213766
127058
379362
353216
351765
356290
396047
396043
326261
363151
162726
288446
362461
381522
226325
190120
20327